In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from transforms import *
from FISTA_RMS import *
from rm_clean import *
from matplotlib import rc
from performance import *
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
## for Palatino and other serif fonts use:
#rc('font',**{'family':'serif','serif':['Palatino']})
rc('text', usetex=True)

In [2]:
#Constants and scales for lambda2 and phi
c = 2.99792458e8
m = 126 #Number of channels
w_min = 3.6 #wavelength start cm
w_max = 50.0 #wavelength end cm

w2_min = (w_min/100.0)*(w_min/100.0) #Min lambda2 in m^2
w2_max = (w_max/100.0)*(w_max/100.0) #Max lambda2 in m^2

lambda2_ref = (w2_max+w2_min)/2 #Reference lambda2 -> lambda2_0
delta_lambda2 = (w2_max-w2_min)/(m-1) #delta lambda2

lambda2 = np.arange(w2_min, w2_max, delta_lambda2) #lambda2 sampling

delta_phi = 2*np.sqrt(3)/(w2_max-w2_min) # delta phi as Brentjens et al - FWHM of the main peak of the RMSF

phi_max = np.sqrt(3)/delta_lambda2 #Phi max as Brentjens et al

times = 4

phi_r = delta_phi/times #The resolution is 4 times less than the FWHM of the main peak of the RMSF

temp = np.int(np.floor(2*phi_max/phi_r)) #we calculate the number of grid points N
n = temp-np.mod(temp,32) #We make sure that n is a even number to use the wavelet transform

phi_r = 2*phi_max/n; 
phi = phi_r*np.arange(-(n/2),(n/2), 1) #phi sampling

In [3]:
# Construct the sensing matrices
ft_matrix = np.zeros((m,len(phi))) + 1j*np.zeros((m,len(phi)))
ift_matrix = np.zeros((len(phi),m)) + 1j*np.zeros((len(phi),m))

for i in range(m):
    for j in range(-int(n/2),int(n/2)):
        ft_matrix[i,j+int(n/2)] = 2*1j*j*phi_r*lambda2[i]


for i in range(-int(n/2),int(n/2)):
    for j in range(m):
        ift_matrix[i+int(n/2),j] = -2*1j*j*phi_r*lambda2[j]
        
FT = np.exp(ft_matrix)
IFT = np.exp(ift_matrix)/n

print(FT.shape)
print(IFT.shape)

(126, 480)
(480, 126)


In [8]:
coherenceFT = coherence(FT)

In [9]:
print(coherenceFT)

0.005557702956762655


In [10]:
analytical_coherence = np.sqrt(1/n)
print(analytical_coherence)

0.04564354645876384
